<a href="https://colab.research.google.com/github/willianrocha/covid-19_vaccine/blob/main/notebooks/COVID_19_Data_Clean_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Campanha Nacional de Vacinação contra Covid-19

Os dados a seguir foram extraídos da página [opendatasus](https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao) sobre os dados da vacinação contra o COVID-19 no dia 05/06/2021.

Os campos do dataset são descritos no [dicionário de dados](https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/38ead83d-b115-4219-852e-7244792bc311) extraídos da página do datasus.

## Manipulação inicial

Dado ao tamanho dos dados de vacinação (~33 GB), as manipulações inicias serão feitas localmente dada a limitação do Google Colab.

In [7]:
import pandas as pd
git_hub_raw = 'https://raw.githubusercontent.com/willianrocha/covid-19_vaccine/main/'
raw_input_header_path = "raw_files/opendatasus_covid19_05jun2020.csv"
pd.read_csv(f"{git_hub_raw}{raw_input_header_path}", sep=';', index_col=0)

,paciente_id,paciente_idade,paciente_datanascimento,paciente_enumsexobiologico,paciente_racacor_codigo,paciente_racacor_valor,paciente_endereco_coibgemunicipio,paciente_endereco_copais,paciente_endereco_nmmunicipio,paciente_endereco_nmpais,paciente_endereco_uf,paciente_endereco_cep,paciente_nacionalidade_enumnacionalidade,estabelecimento_valor,estabelecimento_razaosocial,estalecimento_nofantasia,estabelecimento_municipio_codigo,estabelecimento_municipio_nome,estabelecimento_uf,vacina_grupoatendimento_codigo,vacina_grupoatendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataaplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem
document_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
d69492e7-2c28-4892-9aac-0a378206c968-i0b0,7518e34074e7262958c5ca15dddf901de351c253565ee1...,77,1943-09-03,M,1,BRANCA,420480,10,CURITIBANOS,BRASIL,SC,89520,B,2302438,PREFEITURA MUNICIPAL DE CURITIBANOS,POSTO DE ATEND MATERNO INFANTIL,420480,CURITIBANOS,SC,0,NaN,NaN,NaN,210068,SERUM INSTITUTE OF INDIA LTD,NaN,2021-03-20,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-03-24T12:31:25.000Z,NaN
00008936-722c-4329-a8f8-49f914c14231-i0b0,fa0290484b076c2b4df18119863be049cf6ebd9690184e...,77,1944-02-21,M,3,PARDA,110020,10,PORTO VELHO,BRASIL,RO,76820,B,213241,MUNICIPIO DE PORTO VELHO,CENTRAL DE REDE DE FRIO PORTO VELHO,110020,PORTO VELHO,RO,204,Pessoas de 75 a 79 anos,2.0,Faixa Etária,210017,FUNDACAO BUTANTAN,Organization/61189445000156,2021-03-23,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-23T12:59:53.000Z,NaN
5af902c3-caff-4a1a-8c8e-efd78c71fb73-i0b0,8e37fba588e2e90a219427b9a5d9c58127a73eb8b4df90...,75,1945-04-10,M,1,BRANCA,510675,10,PONTES E LACERDA,BRASIL,MT,78250,B,2394456,PREFEITURA MUNICIPAL DE PONTES E LACERDA,CENTRO DE SAUDE DE PONTES E LACERDA,510675,PONTES E LACERDA,MT,301,Pessoas de 60 nos ou mais Institucionalizadas,3.0,Pessoas de 60 anos ou mais institucionalizadas,210086,FUNDACAO BUTANTAN,Organization/61189445000156,2021-03-30,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-03T13:07:42.000Z,NaN
7fa5874c-7bf6-47ba-ab4b-50a95d8ee472-i0b0,11220f7797eb222c83c82fbff09d2af82aebc6ddab6c6b...,90,1930-06-13,F,1,BRANCA,330100,10,CAMPOS DOS GOYTACAZES,BRASIL,RJ,28051,B,6298109,SECRETARIA MUNICIPAL DE SAUDE DE CAMPOS DOS GO...,SECRETARIA MUNICIPAL DE SAUDE DE CAMPOS DOS GO...,330100,CAMPOS DOS GOYTACAZES,RJ,205,Pessoas de 80 anos ou mais,2.0,Faixa Etária,210011A,FUNDACAO BUTANTAN,Organization/61189445000156,2021-02-18,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-17T19:47:19.000Z,NaN


Como visto acima, há muitas informações que deixam o dataset imenso e difícil de trabalhar. Apenas algumas colunas serão extraídas.

### Vacinas

Como visto no data set original, apenas algumas informações serão utilizadas. Uma delas é a vacina.

#### Nome da vacina
Vamos observar os campos `vacina_codigo` e `vacina_nome`. De acordo com o dicionário, temos:

Ordem  | Campo | Descrição 
-------|-------|----------
30 | vacina_codigo | Código da Vacina
31 | vacina_nome | Nome da vacina/produto

O código da vacina é um número muito mais fácil de manipular e de menor custo de armazenamento. Portanto apenas esta coluna será utilizada no dataset final. Para termos todos os tipos de vacina, foi utilizado o seguinte comando:
```
cut part-00000-90c30073-76b7-418d-9907-e5c0982f9417-c000.csv -d ';' -f30,31 | sed 's/"//g' | sort -u > vacine_list.csv
```

E como resultado, temos:

vacina_codigo | vacina_nome
-------|-------
33 | Pendente Identificação
85 | Vacina Covid-19 - Covishield
86 | Covid-19-Coronavac-Sinovac/Butantan
87 | Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer
88 | Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag
89 | Covid-19-AstraZeneca

Com isso, temos as vacinas sendo aplicadas no Brasil. Uma observação importante são para as vacinas `85` e `89`. Embora tenham códigos diferentes, elas são a mesma vacina! Conhecida como "AstraZenica". O que as diferencia são a origem de fabricação[[1]](https://www.canada.ca/en/health-canada/services/drugs-health-products/covid19-industry/drugs-vaccines-treatments/vaccines/astrazeneca.html). Enquanto a vacina `85` Covishield é feita pelo Verity Pharmaceuticals e Serum Institute of India (SII), ambas na India, e `89` AstraZeneca é feita na China, pelo laboratório Wuxi Biologics[[2]](https://www.uol.com.br/vivabem/noticias/redacao/2021/02/06/ifa-o-que-e-o-ingrediente-farmaceutico-ativo-usado-na-producao-de-vacinas.htm?cmpid=copiaecol).

#### Dose aplicada

Extraído do dicionário:

Ordem  | Campo | Descrição 
-------|-------|----------
29 | vacina_descricao_dose | Descrição da dose

Como feito anteriormente, foi extraído apenas os valores únicos. Temos alguns valores inexperados, como 3ª dose e algumas entradas no dataset original apresentam apenas "dose", não sendo possível distringuir qual tipo de dose é. Considerando a quantidade de aparições de dose, podemos considerar como erro de entrada.

vacina_descricao_dose | Quantidade
------- | ------- 
1ª Dose | 45397507 
2ª Dose | 20810315
Única | 2575
Dose | 245
3ª Dose | 174

Comando para extração usado:
```
cut part-00000-90c30073-76b7-418d-9907-e5c0982f9417-c000.csv -d ';' -f29 | sed 's/"//g' | sort -u > vacine_dose.csv
```

Como esta coluna no dataset original possui caracteres irrelevantes, esta coluna foi modificada para ter apenas códigos sobre qual dose aplicada. Isso acaba economizando no tamanho final do csv.

Descrição | Código
------- | ------- 
1ª Dose | 1 
2ª Dose | 2
3ª Dose | 3
Única | 4
Dose | 5


### Categoria e Grupo
O Dicionário de dados referencia a categoria como um termo macro. Por exemplo, o grupo `2` "Faixa Etária". Enquanto que o grupo detalha melhor de qual Faixa Etária pertence, como "Pessoas de 80 anos ou mais". A lista de grupos é muito extensa e foi omitida.

Ordem  | Campo | Descrição 
-------|-------|----------
21 | vacina_grupoatendimento_codigo | Código do grupo de atendimento do qual pertence o vacinado
21 | vacina_grupoatendimento_nome | Nome do grupo de atendimento do qual pertence o vacinado
21 | vacina_categoria_codigo | Código da categoria
21 | vacina_categoria_nome | Nome da categoria

Abaixo, as categorias encontradas no dataset.

vacina_categoria_codigo | vacina_categoria_nome
------- | ------- 
1 | Comorbidades
2 | Faixa Etária
3 | Pessoas de 60 anos ou mais institucionalizadas
4 | Forças Armadas (membros ativos)
5 | Forças de Segurança e Salvamento
6 | Povos e Comunidades Tradicionais
7 | Povos Indígenas
8 | Trabalhadores da Educação
9 | Trabalhadores de Saúde
10 | Trabalhadores de Transporte
11 | Pessoas com Deficiência
12 | Pessoas em Situação de Rua
13 | Trabalhadores Portuários
14 | Funcionário do Sistema de Privação de Liberdade
15 | População Privada de Liberdade
16 | Trabalhadores Industriais
21 | 
25 | 
94 | 
114 | 


```
cut part-00000-90c30073-76b7-418d-9907-e5c0982f9417-c000.csv -d ';' -f23,24 | sed 's/"//g' | sort -u > vacine_category.csv
cut part-00000-90c30073-76b7-418d-9907-e5c0982f9417-c000.csv -d ';' -f21,22 | sed 's/"//g' | sort -u > vacine_category_group.csv
```

### Idade, sexo biológico, estado e município

Além dos dados acima, dados como idade, sexo e localização serão utilizados.

### Dados extraídos

Os dados extraídos do dataset original são:


*   paciente_idade
*   paciente_enumsexobiologico
*   estabelecimento_uf
*   vacina_categoria_codigo
*   vacina_dataaplicacao
*   vacina_descricao_dose
*   vacina_codigo

E como descrito em vacinas, a coluna `vacina_descricao_dose` foi retrabalhada com o seguinte script:

```
import sys

doses = {
    '1\xc2\xaa\xc2\xa0Dose' : '1',
    '2\xc2\xaa\xc2\xa0Dose' : '2',
    '3\xc2\xaa\xc2\xa0Dose' : '3',
    '\xc3\x9anica\xc2\xa0' : '4',
    'Dose\xc2\xa0' : '5',
    'vacina_descricao_dose' : 'vacina_descricao_dose',
}

for line in sys.stdin:
    full_line = line.split(";")
    d = full_line[5]
    try:
        full_line[5] = doses[d]
        print(",".join(full_line).strip())
    except KeyError:
        sys.stderr.write(repr(d))
```
E o comando final utilizado:
```
cut part-00000-90c30073-76b7-418d-9907-e5c0982f9417-c000.csv -d ';' -f3,5,20,23,28,29,30 | sed 's/"//g' | sed -E 's/    //g'| python format_dataset.py > opendatasus_covid19_05jun2020_refined_v2.csv
```
Em seguida compactado:
```
tar -czvf opendatasus_covid19_05jun2020_refined_v2.tar.gz  opendatasus_covid19_05jun2020_refined_v2.csv
```



# Referências

[1] - https://www.canada.ca/en/health-canada/services/drugs-health-products/covid19-industry/drugs-vaccines-treatments/vaccines/astrazeneca.html

[2] - https://www.uol.com.br/vivabem/noticias/redacao/2021/02/06/ifa-o-que-e-o-ingrediente-farmaceutico-ativo-usado-na-producao-de-vacinas.htm?cmpid=copiaecol